In [1]:
%%time
#Import Libraries
import json
import spotipy
import spotipy.util as util
import pandas as pd
import numpy as np
import time
import random
import csv
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from spotipy.oauth2 import SpotifyClientCredentials

In [81]:
%%time
#assign parameters for authorization
client_id = '122378e2606d48a7bb61507f17d7e3e9'
client_secret = 'adeed656de8642cc895bb6e09e6b1a1e'
username = 'jofussaures'
scope = 'user-read-playback-state user-library-read user-modify-playback-state'
redirect_uri = 'https://localhost:8080/callback/'

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

#initiates spotipy with token, authorized by user
sp = spotipy.Spotify(auth = token)

Wall time: 306 ms


In [3]:
%%time
#checks to see what Spotify account gave authorization
current_user = sp.current_user()

In [84]:
%%time
#Gets song that is currently playing
current_song1 = sp.current_user_playing_track()

Wall time: 84.2 ms


In [85]:
%%time
#gets the track URI
current_track_uri = current_song1['item']['uri']

Wall time: 0 ns


In [7]:
%%time
uri = []
number_of_tracks = 50
for i in range(0,10000,50):
    all_tracks = sp.current_user_saved_tracks(number_of_tracks, i)
    for item in all_tracks['items']:
        track = item['track']
        uri.append(track['uri'])

In [13]:
%%time
tracks = []
def getTrackFeatures(uri):
    meta = sp.track(uri)
    features = sp.audio_features(uri)

  # meta
    name = meta['name']
    artists = meta['album']['artists'][0]['name']
    year = meta['album']['release_date'][:4]
    trackUri = meta['uri']


  # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    key = features[0]['key']
    liveness = features[0]['liveness']
    mode = features[0]['mode']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    valence = features[0]['valence']

    track = [name, artists, year, acousticness, danceability, energy, instrumentalness, key, 
             liveness, mode, loudness, speechiness, tempo, valence, uri]
    return track

for i in range(len(uri)):
    time.sleep(.005)
    track = getTrackFeatures(uri[i])
    tracks.append(track)
    
df = pd.DataFrame(tracks, columns = ['name', 'artists', 'year', 'acousticness', 'danceability', 'energy', 'instrumentalness',
                                     'key', 'liveness', 'mode','loudness', 'speechiness', 'tempo', 'valence', 'uri'])
df.to_csv("spotify.csv", sep = ',')

In [14]:
%%time
#Import datasheet with pandas
#rdata = rawdata
df = pd.read_csv('spotify.csv')

Wall time: 28.9 ms


In [18]:
%%time
#dfw = dataframe w/o name and artists
da = df['artists']
dn = df['name']
du = df['uri']
dfw=df.drop(columns = ['name','artists', 'uri'])

Wall time: 0 ns


In [19]:
%%time
features = dfw[['acousticness','danceability','energy','instrumentalness','loudness','speechiness','tempo','valence']]

Wall time: 0 ns


In [20]:
%%time
km = KMeans(n_clusters = 4)
km.fit(dfw)
cluster = km.labels_

Wall time: 174 ms


In [21]:
%%time
newdf=dfw.assign(Cluster=cluster)
grouped=newdf.groupby(["Cluster"])
grouped.agg({'acousticness' : 'mean','danceability' : 'mean','energy' : 'mean','instrumentalness' : 'mean','loudness' : 'mean',
             'speechiness' : 'mean','tempo' : 'mean','valence' : 'mean',})

Wall time: 25 ms


,acousticness,danceability,energy,instrumentalness,loudness,speechiness,tempo,valence
Cluster,,,,,,,,
0,0.240916,0.621352,0.624851,0.009765,-6.969323,0.153200,123.167423,0.435589
1,0.210147,0.622656,0.648614,0.013567,-6.412803,0.146805,120.203420,0.432515
2,0.179091,0.650929,0.614137,0.012772,-7.123590,0.155164,118.991776,0.411234
3,0.191187,0.688116,0.639763,0.011369,-6.532625,0.168533,119.943830,0.486716


In [22]:
%%time
newdf_melt=pd.melt(newdf.reset_index(), id_vars =['Cluster'],value_vars=['acousticness','danceability','energy',
                                                                         'instrumentalness','loudness','speechiness','tempo',
                                                                         'valence'],var_name='Metric',value_name='Value')

Wall time: 16.4 ms


In [23]:
%%time
c0 = newdf[newdf['Cluster']==0].reindex()
c1 = newdf[newdf['Cluster']==1].reindex()
c2 = newdf[newdf['Cluster']==2].reindex()
c3 = newdf[newdf['Cluster']==3].reindex()

Wall time: 8.1 ms


In [30]:
%%time
#add artists and song name back into clusters (at end of DataFrame)
#c#a = cluster# w/ artist
#c#f = cluster# w/ artist and name
#c#f = cluster# w/ full data
c0a = c0.assign(artists=df['artists'])
c0an = c0a.assign(name=df['name'])
c0f = c0an.assign(uri=df['uri'])

c1a = c1.assign(artists=df['artists'])
c1an = c1a.assign(name=df['name'])
c1f = c1an.assign(uri=df['uri'])

c2a = c2.assign(artists=df['artists'])
c2an = c2a.assign(name=df['name'])
c2f = c2an.assign(uri=df['uri'])

c3a = c3.assign(artists=df['artists'])
c3an = c3a.assign(name=df['name'])
c3f = c3an.assign(uri=df['uri'])

Wall time: 21.4 ms


In [31]:
%%time
#puts artists and name at beginning of DataFrame
neworder = ['name','artists','year','acousticness','danceability','energy','instrumentalness','key','liveness',
            'loudness','mode','speechiness','tempo','valence','Cluster', 'uri']
c0f = c0f.reindex(columns = neworder)
c1f = c1f.reindex(columns = neworder)
c2f = c2f.reindex(columns = neworder)
c3f = c3f.reindex(columns = neworder)

Wall time: 12 ms


In [32]:
%%time
fourframes = [c0f,c1f,c2f,c3f]
allsongs = pd.concat(fourframes)
allsongs

Wall time: 9.67 ms


,name,artists,year,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,Cluster,uri
812,Can't Be Broken,Lil Wayne,2018,0.1250,0.580,0.577,0.000000,2,0.0860,-5.997,0,NaN,0.2000,99.283,0.153,0,spotify:track:5cGpjfgAvidLy6znwkCxkP
813,Let It Fly (feat. Travis Scott),Lil Wayne,2018,0.0410,0.902,0.819,0.000000,1,0.1060,-3.321,1,NaN,0.3160,89.000,0.745,0,spotify:track:0CBQ0m3mufWH75LTYB3s6h
814,Uproar,Lil Wayne,2018,0.0595,0.743,0.870,0.000000,1,0.2990,-2.188,1,NaN,0.2120,99.079,0.884,0,spotify:track:3wScL5W8H40zzCKN0atfBk
815,Dedicate,Lil Wayne,2018,0.0330,0.879,0.662,0.000000,2,0.7160,-6.162,1,NaN,0.3090,117.975,0.724,0,spotify:track:0Y7di9ZCmB6Elad5WISaPc
816,Don't Cry (feat. XXXTENTACION),Lil Wayne,2018,0.4070,0.618,0.710,0.000000,8,0.1830,-6.066,1,NaN,0.3310,112.014,0.233,0,spotify:track:5BQrp63SHCVf4bzCzJePne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,Magic Shop,BTS,2018,0.1030,0.541,0.661,0.000000,1,0.0779,-5.838,1,NaN,0.0889,149.926,0.285,3,spotify:track:22ryCgQttpV7oCrn7llGRu
401,Walk Man,Tiny Meat Gang,2019,0.3540,0.931,0.699,0.000158,4,0.1210,-4.991,0,NaN,0.0781,94.987,0.400,3,spotify:track:7bDaswZlpaAe8rPQOWHySn
402,Spring Day,BTS,2017,0.0063,0.539,0.846,0.000002,8,0.1300,-3.192,1,NaN,0.0354,106.992,0.460,3,spotify:track:0WNGsQ1oAuHzNTk8jivBKW
403,Just One Day,BTS,2014,0.1750,0.504,0.712,0.000000,9,0.3770,-4.705,1,NaN,0.3790,189.577,0.307,3,spotify:track:4ZSLc2F15wKVPjyLt169tt


In [86]:
%%time
playing_track = allsongs.loc[allsongs['uri'] == current_track_uri]
playing_track

Wall time: 0 ns


,name,artists,year,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,Cluster,uri
1594,Exchange,Bryson Tiller,2015,0.107,0.525,0.433,0.0,6,0.135,-10.598,1,NaN,0.185,160.108,0.276,2,spotify:track:43PuMrRfbyyuz4QpZ3oAwN


In [87]:
%%time
track_cluster = playing_track.iloc[0]['Cluster']
track_cluster

Wall time: 0 ns


2

In [88]:
playing_track_year = playing_track.iloc[0]['year']
playing_track_year

2015

In [89]:
add_to_queue = allsongs[(playing_track_year >= (allsongs['year'] - 5)) & (playing_track_year <= (allsongs['year'] + 5))
        & (track_cluster == allsongs['Cluster'])].sample(10)
add_to_queue

,name,artists,year,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,Cluster,uri
1269,Primadonna,MARINA,2012,0.088400,0.660,0.689,0.000000,4,0.0922,-2.671,0,NaN,0.0337,127.980,0.427,2,spotify:track:5tFMuAw6X2R4NLx97g7n9q
1346,Breathe,G-Eazy,2012,0.000972,0.635,0.458,0.000000,8,0.1810,-9.525,1,NaN,0.0801,76.067,0.071,2,spotify:track:25nmwq15BIGIFF44Koftyy
1442,Not the Same,KYLE,2017,0.341000,0.790,0.606,0.000093,5,0.0697,-5.457,0,NaN,0.0501,117.963,0.514,2,spotify:track:5QyziaJzRpQ3p8tdw1dmdA
1266,Living Single,Big Sean,2016,0.223000,0.714,0.620,0.000000,11,0.1090,-5.045,1,NaN,0.2960,80.109,0.730,2,spotify:track:36xxq7VDZZsGwARsn6Clo7
1230,No. 1 Party Anthem,Arctic Monkeys,2013,0.330000,0.505,0.698,0.088700,0,0.0966,-5.563,1,NaN,0.0297,115.013,0.599,2,spotify:track:5TTGoX70AFrTvuEtqHK37S
1249,HotelSixNine (feat. Solomon da God),Spooky Black,2014,0.233000,0.668,0.244,0.904000,1,0.0981,-21.035,1,NaN,0.1240,109.956,0.104,2,spotify:track:30dF8ON0TT1PO6bXSLB36C
1232,Faygo Dreams,6 Dogs,2017,0.224000,0.748,0.660,0.053000,6,0.1010,-4.704,1,NaN,0.0487,137.033,0.328,2,spotify:track:2BoeDIEzoS9diuyaqqVaDh
1371,Nasty (Who Dat) (feat. Migos),A$AP Ferg,2017,0.020300,0.661,0.636,0.000023,4,0.3780,-4.551,0,NaN,0.1180,179.041,0.428,2,spotify:track:76T8y91xMLWJgfsEsmWGxW
1334,Nights,Frank Ocean,2016,0.420000,0.466,0.548,0.000001,5,0.1130,-9.362,0,NaN,0.1180,89.815,0.423,2,spotify:track:7eqoqGkKwgOaWNNHx90uEZ
1253,Pink Matter,Frank Ocean,2012,0.951000,0.392,0.233,0.000322,11,0.1060,-11.012,0,NaN,0.0587,176.569,0.494,2,spotify:track:1fOkmYW3ZFkkjIdOZSf596


In [90]:
to_queue = add_to_queue["uri"].tolist()

for track in to_queue:
    sp.add_to_queue(track)

In [ ]:
#***search for currentTrackURI() in allsongs.df

#***get currentTrackURI()'s cluster

#***search for X# of songs +-5 years of song && in same cluster
     #return song's URI to list

#for each URI in list, add to queue